In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import math

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('../data/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class LoRALayer(nn.Module):
    def __init__(self, in_features, out_features, rank=4, alpha=1, scale=0.01, use_bias=True):
        super(LoRALayer, self).__init__()
        self.rank = rank
        self.alpha = alpha
        self.count = 0

        # Original weights, with or without bias based on use_bias
        self.W = nn.Linear(in_features, out_features, bias=use_bias)
        
        # Freeze the original weights
        for param in self.W.parameters():
            param.requires_grad = False

        # Low-rank matrices W1 and W2
        self.lora_W1 = nn.Parameter(torch.Tensor(out_features, rank))
        self.lora_W2 = nn.Parameter(torch.Tensor(rank, in_features))

        # Initialize the LoRA matrices
        nn.init.kaiming_uniform_(self.lora_W1, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.lora_W2, a=math.sqrt(5))

        # Scale down the LoRA matrices
        self.lora_W1.data *= scale
        self.lora_W2.data *= scale

    def forward(self, x):
        self.count += 1
        lora_adjustment = self.lora_W1 @ self.lora_W2
        W_lora = self.W.weight + self.alpha * lora_adjustment
        output = F.linear(x, W_lora, self.W.bias if self.W.bias is not None else None)
        return output


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = LoRALayer(n_embd, head_size, use_bias=False)
        self.query = LoRALayer(n_embd, head_size, use_bias=False)
        self.value = LoRALayer(n_embd, head_size, use_bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = LoRALayer(n_embd, n_embd, use_bias=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            LoRALayer(n_embd, 4 * n_embd, use_bias=True),
            nn.ReLU(),
            LoRALayer(4 * n_embd, n_embd, use_bias=True),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = LoRALayer(n_embd, vocab_size, use_bias=True)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.872773 M parameters


In [8]:
# model.load_state_dict(torch.load('../model_weights/dec_9.pth'))

In [19]:
# Assuming pretrained_dict contains your pretrained model weights
pretrained_dict = torch.load('../model_weights/dec_10.pth')

# Initialize your modified model
model = BigramLanguageModel()  # with LoRALayers
model = model.to(device)

# Extract only the matching parts of the pretrained model
new_state_dict = model.state_dict()
for name, param in pretrained_dict.items():
    # Match the original Linear layer weights and biases
    if 'key.weight' in name or 'query.weight' in name or 'value.weight' in name:
        lora_name = name.replace('.weight', '.W.weight')  # Adjust for LoRALayer naming
        if lora_name in new_state_dict:
            new_state_dict[lora_name].copy_(param)
    elif 'key.bias' in name or 'query.bias' in name or 'value.bias' in name:
        lora_name = name.replace('.bias', '.W.bias')  # Adjust for LoRALayer naming
        if lora_name in new_state_dict:
            new_state_dict[lora_name].copy_(param)
    else:
        if name in new_state_dict and param.size() == new_state_dict[name].size():
            new_state_dict[name].copy_(param)

# Load the updated state dict into your model
model.load_state_dict(new_state_dict, strict=False)


<All keys matched successfully>

In [21]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))


B; eel abukl ton, him lomy ayd fe witin,
And I Gry, higo m fasstou s,--
Ave th tere bllear ctherld pe isenookpave I't O--l Se fZo prinetowed mman
Bers
The wouse sis is ith shee
Th! thed Cin d ym o thandofy cowe be mand hilllld hat:
3 therell mor, surend dshencead!
Meng lllll unes gs y areased pis wiadiley meawhe ain
n: aprthu ouril has be.

Wlllsll iveke ckshada my s,
Dothe s o he dous.


ERCNCES:

&OSibLarerarer shof my ovincig he thir we:
Feait fy,
y:
Rr he wik slll fofud as mowind furcemanel my hel,
I blwa.
YAne Ast f at-NG, he theimeaigck'd meghor A.


Fo haverdriesver , m be mae wind, I I haro'd ; woveator:
:

Wewre my har f oru Y matheerrd I hay ssllullfis beebp dth me we weirgoon,
Tof t m ncut; thames, wier,

And me y shrofarkered wncende is, thacod bickit s is,
Angr, b
An fateso wapowht win heBacleg, yerm ay y blaithe.



PEishegun I'le d ICLe:
Tom bllavey t.
YSouy wirimalld h E-lirith Cere;
P:
O! hy athar by crincendive k'lly toby:
Andish, t htherde blllothes I ilfay wowi isp

In [6]:
from time import time

start_time = time()
max_iters = 100
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % (max_iters // 10) == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
end_time = time()

total_time = end_time - start_time

total_time

step 0: train loss 4.2821, val loss 4.2836
step 10: train loss 3.2224, val loss 3.2639
step 20: train loss 2.9069, val loss 2.9274
step 30: train loss 2.7111, val loss 2.7327
step 40: train loss 2.6244, val loss 2.6232
step 50: train loss 2.5785, val loss 2.5712
step 60: train loss 2.5583, val loss 2.5457
step 70: train loss 2.5247, val loss 2.5282
step 80: train loss 2.5301, val loss 2.5235
step 90: train loss 2.4993, val loss 2.5070
step 99: train loss 2.4828, val loss 2.4970


66.53520631790161

In [22]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))


Sh's con; buemeh tfrs ftitewspim the'set,
Anequink ws henoo inck be kere ars, my.

Fore:
Laushalal:
Seare win af ut villsthengot st kn?
NG oand, Cay hatery haiofuge hamy odisy

Thather I ormell, t vatille al s bom. grellym, wibase n yoous sourgie herer cachar
Frul ancif oungs sss nd,
NDLawat it ts lend Ye wsprlirow hey mevex ithe se&.

Doffalllf thathestr'l tht we hea win peder pre wintar Pagchyooul we,
Ka od,, l peanenenkevanck unatho' ap I gerd t b ot cagathxpl toning thanl o by

Alid mathou dly ngelllldrkeloiomnte fe, ors hy bn s:
I watgad avend sithallll g.


Lof th I sar Thampe sthes mat bo ameacisivel my nevet gn cusplthi,

What m, wir ce miwathke hill durpllbatey rveary.


MAIZ-BRIS:

CEfy INY
MHairl hove maaasu thellun T&K: t ssthorel Soue.
P he ss REe te been, Hintis, GI rf; jeo n wa thiqpothisins ng Yergunce!
Hers hithes ows.
Tes orry hy, winthin WProoves s wamethe myout t hin lde thillce!

Wonche s GEf RE-POL:
TSZRiveil RINOYopank me w bu ofes, n Sonme at d
Alan; sun ced yo